In [1]:
import numpy as np
import pandas as pd

In [2]:
def update_missing_feature_list(df):
    """
    Perform updating of Feature Information 'missing_or_unknown' so that it can be more easily used.
    Remove the brackets and convert numerical strings to integers. 
    Put updated values in a new column.
    
    INPUT: Feature Information DataFrame
    OUTPUT: FEature Information DataFrame with updated 
    """
    
    feat_info = df.copy()
    feat_info['missing'] = feat_info['missing_or_unknown'].copy()
    characters = ['', 'X', 'XX'] # These are the non numerical strings 

    # iterate through feat_info row by row and create a new list from the original one. 
    for idx in range(feat_info.shape[0]):
        
        # create a list from 'missing_or_unknown' at row idx and remove the brackets
        temp_list  = feat_info.iloc[idx]['missing'][1:-1].split(sep=',')                                
        temp_list2 = []                   
        for c in temp_list:               # iterate through the list 
            if c in characters:           # if 'X' or 'XX', add to new list as is
                temp_list2.append(c)
            else:                         # if not, it's a number and needs
                temp_list2.append(int(c)) #    to be converted to int before added to new list
        temp_list = temp_list2
    
        # Update 'missing' with the new list
        feat_info.at[idx, 'missing'] = temp_list

    return feat_info

In [7]:
def replace_missing_with_nan(data_df, feature_df):
    """
    Replace missing information in data_df with NaNs. 
    Missing information is derived from feature_df['missing'].
        
    INPUT: Data DataGrame, Feature Information DataFrame
    OUTPUT: Data DataFrame with missing informaiton replaced by NaNs. 
    """
    
    data      = data_df.copy()
    feat_info = feature_df.copy()
    data_cols = data.columns

    # iterate through all the features (the columns) in data_df
    # replace any values that correspond to feature_df['missing'] with NaN. 
    for col in data_cols:
        temp_feature = data[col].copy()
        temp_missing = feat_info.loc[feat_info['attribute'] == col]['missing'].array
        temp_feature[temp_feature.isin(temp_missing[0])] = np.nan
        data[col] = temp_feature
        
    return data

In [24]:
def drop_columns_from_data(data_df):
    """
    Calculate the number and the percentage of NaNs for each column (feature) of data_df. 
    Drop columns where '% NaNs' is greater than a threshold. 
    33% is the default threshold. 
    An algorithm to determine the appropriate threshold could be developed. 
        
    INPUT: Data DataFrame, threshold
    OUTPUT: Data DataFrame with columns where '% NaNs' greater than threshold are dropped 
    """
    
    data = data_df.copy()
    missing_col_data = {}
    COL_THRESHOLD    = 33
    
    # create a dictionary of columns and number of NaNs
    for col in data.columns:        
        missing_col_data[col] = data[col].isnull().sum() 

    # use that dictionary to derive the percent of NaNs for each feature    
    missing_features = pd.DataFrame.from_dict(missing_col_data, orient='index')
    missing_features.rename(columns={0:'# NaNs'}, inplace=True)
    missing_features['% NaNs'] = 100 * missing_features['# NaNs'] / data.shape[0]
    # determine which features to drop
    features_to_drop = missing_features[missing_features['% NaNs'] > COL_THRESHOLD].index.tolist()
    data.drop(features_to_drop, axis=1, inplace=True)
    
    return data, features_to_drop

In [28]:
def drop_rows_from_data(data_df):
    """
    Calculate the number and the percentage of NaNs for each row (observation) of data_df. 
    Drop columns where '% NaNs' is greater than a threshold. 
    5% is the default threshold. 
    An algorithm to determine the appropriate threshold could be developed. 
        
    INPUT:  Data DataFrame
    OUTPUT: Data DataFrame with rows where '% NaNs' greater than threshold are dropped 
    """
    data = data_df.copy()
    missing_col_data = {}
    ROW_THRESHOLD    = 5
    
    data['# NaNs']  = data.isnull().sum(axis=1)
    data['% NaNs']  = 100 * data['# NaNs'] / (data.shape[1] - 1)
    data_lower      = data[data['% NaNs'] < ROW_THRESHOLD]
    
    #print (data_lower.shape, data_lower['% NaNs'].max())
    
    data_lower.drop('# NaNs', axis=1, inplace=True)
    data_lower.drop('% NaNs', axis=1, inplace=True)
    
    return data_lower 

In [22]:
def re_encode_categorical_features(data_df, feature_df, features_drop):
    """
    Performs one hot encoding of remaining columns (features) in data_df that are categorical.
    Drops the original column (feature)
        
    INPUT:  Data DataFrame, Feature Information DataFrame, list of features that have been dropped
    OUTPUT: Data DataFrame with categorical data re-encoded with one hot encoding.  
    """
    
    data              = data_df.copy()
    feat_info         = feature_df.copy()
    feat_info_reduced = feat_info.copy()
    features_to_drop  = features_drop.copy()
    
    # this creates a list of features minus the ones that were dropped earlier
    feat_info_reduced = feat_info_reduced[~feat_info_reduced['attribute'].isin(features_to_drop)]
    # this is a list of categorical features from the reduced list
    categorical       = feat_info_reduced[feat_info['type']=='categorical']['attribute'].tolist()
    
    # iterate through the categorical features and one hot encode as necessary
    for feature in categorical: 
        values = data[feature].unique()
        if (values.size > 2) | (type(values[0]) == str):                          # if string or multi-level
            x = pd.get_dummies(data[feature], prefix=[feature])                   #    one hot encode
            data[x.columns] = x                                                   #    concatenate
            data.drop(feature, axis=1, inplace=True)                              #    drop the original feature

    return data    

In [34]:
def re_encode_mixed_features(data_df, feature_df, features_drop):
    """
    Performs specialized encoding of two features in data_df: PRAEGENDE_JUGENDJAHRE and CAMEO_INTL_2015.
    Drops the original feature columns. 
        
    INPUT:  Data DataFrame, Feature Information DataFrame, list of features that have been dropped
    OUTPUT: Data DataFrame with specific mixed data re-encoded with specialized encoding.  
    """
    
    data              = data_df.copy()
    feat_info         = feature_df.copy()
    feat_info_reduced = feat_info.copy()
    features_to_drop  = features_drop.copy()
    
    # this is a mappinf of the feature PRAEGENDE_JUGENDJAHRE
    PRAEGENDE_JUGENDJAHRE_map = {0:[0,0],   1:[1,1], 2:[2,1],
                             3:[1,2],   4:[2,2], 5:[1,3],
                             6:[2,3],   7:[2,3], 8:[1,4],
                             9:[2,4],  10:[1,5],11:[2,5],
                             12:[1,5], 13:[2,5],14:[1,6],
                             15:[2,6]
                            }
    
    # this creates a list of features minus the ones that were dropped earlier
    feat_info_reduced = feat_info_reduced[~feat_info_reduced['attribute'].isin(features_to_drop)]
    # this is a list of cmixed features from the reduced list
    mixed = feat_info_reduced[feat_info['type']=='mixed']['attribute'].tolist()
    
    # address PRAEGENDE_JUGENDJAHRE
    PRAEGENDE_JUGENDJAHRE_map = {0:[0,0],   1:[1,1], 2:[2,1],
                             3:[1,2],   4:[2,2], 5:[1,3],
                             6:[2,3],   7:[2,3], 8:[1,4],
                             9:[2,4],  10:[1,5],11:[2,5],
                             12:[1,5], 13:[2,5],14:[1,6],
                             15:[2,6]
                            }
    # set NaNs to 0
    data['PRAEGENDE_JUGENDJAHRE'].fillna(value=0, inplace=True)

    # create two new features and map them using PRAEGENDE_JUGENDJAHRE_map
    data['PRAEGENDE_JUGENDJAHRE_movement'] = \
         data['PRAEGENDE_JUGENDJAHRE'].map(PRAEGENDE_JUGENDJAHRE_map).map(lambda x: x[0])
    data['PRAEGENDE_JUGENDJAHRE_decade'] = \
         data['PRAEGENDE_JUGENDJAHRE'].map(PRAEGENDE_JUGENDJAHRE_map).map(lambda x: x[1])
    # for this exercise, remove the original feature
    data.drop('PRAEGENDE_JUGENDJAHRE', axis=1, inplace=True)
    
    # address CAMEO_INTL_2015
    # split the 10's from the 1's for the new features.
    
    # set NaNs to 99
    data['CAMEO_INTL_2015'].fillna(value=99, inplace=True)

    # create two new features by splitting the two digit numeric. 
    # convert the series in to strings, use .str to slice them, then turn them back into ints. 
    data['CAMEO_INTL_2015']            = data['CAMEO_INTL_2015'].astype(str)
    data['CAMEO_INTL_2015_wealth']     = data['CAMEO_INTL_2015'].str[0].astype(int)
    data['CAMEO_INTL_2015_lifestage']  = data['CAMEO_INTL_2015'].str[1].astype(int)
    data.drop('CAMEO_INTL_2015', axis=1, inplace=True)
    
    # the rest of the mixed categories are left as is
    
    return data

In [32]:
azdias_raw                      = pd.read_csv('Udacity_AZDIAS_Subset.csv', delimiter=';')
feature_info_raw                = pd.read_csv('AZDIAS_Feature_Summary.csv', delimiter=';')

feature_info_clean              = update_missing_feature_list(feature_info_raw)
azdias_clean                    = replace_missing_with_nan(azdias_raw, feature_info_clean)

azdias_clean, dropped_features  = drop_columns_from_data(azdias_clean)
azdias_clean                    = drop_rows_from_data(azdias_clean)

azdias_clean                    = re_encode_categorical_features(azdias_clean, feature_info_clean, dropped_features)
azdias_clean                    = re_encode_mixed_features(azdias_clean, feature_info_clean, dropped_features)

C:\Users\wc5257\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\wc5257\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [33]:
azdias_clean.head()

,ALTERSKATEGORIE_GROB,ANREDE_KZ,FINANZ_MINIMALIST,FINANZ_SPARER,FINANZ_VORSORGER,FINANZ_ANLEGER,FINANZ_UNAUFFAELLIGER,FINANZ_HAUSBAUER,GREEN_AVANTGARDE,HEALTH_TYP,...,['CAMEO_DEU_2015']_8D,['CAMEO_DEU_2015']_9A,['CAMEO_DEU_2015']_9B,['CAMEO_DEU_2015']_9C,['CAMEO_DEU_2015']_9D,['CAMEO_DEU_2015']_9E,PRAEGENDE_JUGENDJAHRE_movement,PRAEGENDE_JUGENDJAHRE_decade,CAMEO_INTL_2015_wealth,CAMEO_INTL_2015_lifestage
1,1.0,2,1,5,2,5,4,5,0,3.0,...,0,0,0,0,0,0,1,6,5,1
2,3.0,2,1,4,1,2,3,5,1,3.0,...,0,0,0,0,0,0,2,6,2,4
4,3.0,1,4,3,4,1,3,2,0,3.0,...,0,0,0,0,0,0,1,4,4,3
5,1.0,2,3,1,5,2,2,5,0,3.0,...,0,0,0,0,0,0,1,2,5,4
6,2.0,2,1,5,1,5,4,3,0,2.0,...,0,0,0,0,0,0,1,5,2,2
